In [249]:
from icecube import dataio, dataclasses, simclasses, phys_services
import os, sys
import numpy as np
import matplotlib as mpl
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
import timeit as time
import math
from datetime import datetime

In [250]:
plt.rcParams.update({'font.size': 20})
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
mpl.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
mpl.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'

mpl.rcParams['hist.bins'] = 100
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['patch.linewidth'] = 2.5
mpl.rcParams['grid.alpha'] = 0.2

In [251]:
files_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999'
file_list = os.listdir(files_dir)
infile_name = os.path.join(files_dir, file_list[0])
print(infile_name)

/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/83.i3.bz2


In [111]:
infile = dataio.I3File(infile_name)
frame = infile.pop_physics()
print(frame)

[ I3Frame  (Physics):
  'AnalysisMuons' [Physics] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (174)
  'BackgroundCLSim_intermediatePhotons' [DAQ] ==> I3Map<ModuleKey, I3Vector<I3Photon> > (3167)
  'BackgroundI3MCPESeriesMap_0.990' [DAQ] ==> I3Map<OMKey, vector<I3MCPE> > (90)
  'BackgroundI3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (1786)
  'BackgroundI3MCTree_preMuonProp' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (918)
  'BackgroundMMCTrackList' [DAQ] ==> I3Vector<I3MMCTrack> (304)
  'BeaconLaunches' [DAQ] ==> I3Map<OMKey, vector<I3DOMLaunch> > (46)
  'CalibratedWaveformRange' [DAQ] ==> I3TimeWindow (48)
  'CascadeFilter_13' [Physics] ==> I3PODHolder<bool> (27)
  'CorsikaInteractionHeight' [DAQ] ==> I3PODHolder<double> (36)
  'CorsikaWeightMap' [DAQ] ==> I3Map<string, double> (482)
  'DCAnalysisHits' [Physics] ==> I3PODHolder<double> (36)
  'DCNHits' [Physics] ==> 

In [112]:
track_list = frame['MMCTrackList']
mctree     = frame['I3MCTree']


In [113]:
# Let's find the muon
print(track_list)


[I3MMCTrack = [
 (xi, yi, zi, ti, Ei) = (-115.176 ,542.973 ,800 ,7702.91 ,559.519)
 (xc, yc, zc, tc, Ec) = (-29.4748 ,181.081 ,-77.554 ,10882.1 ,267.507)
 (xf, yf, zf, tf, Ef) = (41.079 ,-116.846 ,-800 ,13499.4 ,60.5039)
 Elost = 499.015
 Particle = [ I3Particle MajorID : 13469127016857373990
             MinorID : 846
              Zenith : 0.400847
             Azimuth : 1.80333
                   X : -227.484
                   Y : 1017.22
                   Z : 1949.99
                Time : 3536.68
              Energy : 1054.01
               Speed : 0.299792
              Length : 3125.59
                Type : MuPlus
        PDG encoding : -13
               Shape : StartingTrack
              Status : NotSet
            Location : InIce
]]
]


In [114]:
print('Injection point of the muon', muon_track.particle.pos)
print('Length that the muon traveled in total', muon_track.particle.length)

# Let's calculate the distance the muon traveled *inside the 800m volume*
entering_large_cylinder = dataclasses.I3Position(muon_track.xi, muon_track.yi, muon_track.zi)
print('Point where it enters the 800m volume', entering_large_cylinder)
d_injection_cylinder = (muon_track.particle.pos - entering_large_cylinder).magnitude
print('Distance from injection to 800m volume', d_injection_cylinder)
d_inside = muon_track.particle.length - d_injection_cylinder

# Ionization energy loss
econstant = 1/4.5 # GeV/m
e_ion = econstant*d_inside
print('Ionization losses', e_ion)

# Sum all the stochastics
daughters = mctree.get_daughters(muon_track.particle)
e_stochastic = 0
for d in daughters:
    e_stochastic += d.energy

print('Stochastic losses', e_stochastic)
print('Total E lost inside volume', e_stochastic+e_ion)
print('Energy available upon entering volume', muon_track.Ei)
print('This does not add up very nicely, why?')



Injection point of the muon I3Position(-1680.74,-809.682,1949.81)
Length that the muon traveled in total 3212.2136875364995
Point where it enters the 800m volume I3Position(-711.279,-366.173,429.537)
Distance from injection to 800m volume 1856.8167589096295
Ionization losses 301.19931747263774
Stochastic losses 0
Total E lost inside volume 301.19931747263774
Energy available upon entering volume 385.1886443591824
This does not add up very nicely, why?


In [116]:

print('This is a good special example. This muon actually left the volume. It did not decay inside.')
# This function can be used to move along a track
# I'm taking the muon, and moving along a track a distance equal to its length
# Then I'm printing the end point
print(muon_track.particle.shift_along_track(muon_track.particle.length))
print('That end point is outside the detector')
print('This muon entered and left. That is fine, but our energy accounting will not work as nicely')
print('There are probably some energy losses outside as it left, that we did not account for')
print('Still, we only care about stochastic energy losses INSIDE the 500m volume, so the fact the muon leaves is fine')

This is a good special example. This muon actually left the volume. It did not decay inside.
I3Position(-3.61592,-42.4311,-680.195)
That end point is outside the detector
This muon entered and left. That is fine, but our energy accounting will not work as nicely
There are probably some energy losses outside as it left, that we did not account for
Still, we only care about stochastic energy losses INSIDE the 500m volume, so the fact the muon leaves is fine


In [117]:
frame = infile.pop_physics()

track_list = frame['MMCTrackList']
mctree     = frame['I3MCTree']

muon_track = track_list[0]

In [118]:

# Check where it decayed
print(muon_track.particle.shift_along_track(muon_track.particle.length))

print(track_list) 

print('Injection point of the muon (X,Y,Z)', muon_track.particle.pos)
print('Length that the muon traveled in total (Length)', muon_track.particle.length)

# Let's calculate the distance the muon traveled *inside the 800m volume*
entering_large_cylinder = dataclasses.I3Position(muon_track.xi, muon_track.yi, muon_track.zi)
print('Point where it enters the 800m volume', entering_large_cylinder)
distance_from_injection_cylinder_to_800m = (muon_track.particle.pos - entering_large_cylinder).magnitude
print('Distance from injection to 800m volume', distance_from_injection_cylinder_to_800m)
d_inside = muon_track.particle.length - distance_from_injection_cylinder_to_800m

# Ionization energy loss
econstant = 1/4.5 # GeV/m
e_ion = econstant * d_inside #GeV
print('Ionization losses', e_ion)

# Sum all the stochastics
daughters = mctree.get_daughters(muon_track.particle)
e_stochastic = 0
for d in daughters:
    e_stochastic += d.energy

print('Stochastic losses', e_stochastic)
print('Total E lost inside 800m volume', e_stochastic+e_ion)
print('Energy available upon entering volume', muon_track.Ei)



I3Position(327.273,-143.257,-1233.63)
[I3MMCTrack = [
 (xi, yi, zi, ti, Ei) = (-497.684 ,-268.444 ,800 ,8350.94 ,1090.31)
 (xc, yc, zc, tc, Ec) = (-144.084 ,-214.785 ,-71.6707 ,11493.7 ,762.833)
 (xf, yf, zf, tf, Ef) = (151.367 ,-169.951 ,-800 ,14119.7 ,332.442)
 Elost = 757.869
 Particle = [ I3Particle MajorID : 13469127016857373990
             MinorID : 50557
              Zenith : 0.389355
             Azimuth : 3.29219
                   X : -964.189
                   Y : -339.236
                   Z : 1950
                Time : 4204.52
              Energy : 2048.32
               Speed : 0.299792
              Length : 3441.19
                Type : MuPlus
        PDG encoding : -13
               Shape : StartingTrack
              Status : NotSet
            Location : InIce
]]
]
Injection point of the muon (X,Y,Z) I3Position(-964.189,-339.236,1950)
Length that the muon traveled in total (Length) 3441.188043537651
Point where it enters the 800m volume I3Position(-497.684,-2

In [186]:
#sorry, one more question

#so in this way I need to establish values for r = 500 and z = (-500;500)

#for this I use x,y,z from which frame? 
# or I just establish them?
print('Entrance point of the particle (x,y,z)', dataclasses.I3Position(muon_track.xi, muon_track.yi, muon_track.zi))
print('Length that the muon traveled in 800 (Length)', d_inside)

daughters = mctree.get_daughters(muon_track.particle)


e_stochastic = 0
for d in daughters:
    position=d.pos
    if np.sqrt((position.x)**2 + (position.y)**2) <= 500.0 and  position.z>=-500.0 and position.z <= 500.0:
        e_stochastic += d.energy
print('Stochastic losses',e_stochastic)
    
    

# Let's calculate the distance the muon traveled *inside the 500m volume*
#entering_small_cylinder = (np.sqrt((x)**2 + (y)**2)==500.0), z ==500.0 #how can I obtain these valeus
#print('Point where it enters the 500m volume', entering_small_cylinder)

#distance_from_injection_800cylinder_to_500m = (muon_track.particle.pos - entering_small_cylinder).magnitude
#print('Distance from injection 800 to 500m volume', distance_from_injection_800cylinder_to_500m)
#d_inside500 = muon_track.particle.length - distance_from_injection_800cylinder_to_500m




Entrance point of the particle (x,y,z) I3Position(-497.684,-268.444,800)
Length that the muon traveled in 800 (Length) 2198.153944077246
Stochastic losses 117.9135687086868


In [255]:
frame_arr = []; mctree = []
track_list = []

for i in range(0, 2):
    files_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/' + str(i) + '.i3.bz2' # path to i3 file
    with dataio.I3File('/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/'+ str(i) + '.i3.bz2') as infile:
        for frame in infile:
            if infile.stream.id != 'P': continue
            frame_arr.append(frame)
            mctree.append(frame['I3MCTree']) 
            track_list.append(frame['MMCTrackList'])

In [261]:
num_muons = []

for it in range(len(track_list)):
    num_muons[it] = len(track_list[it])
    
plt.hist(num_muons, np.arange(1,25))
print(track_list[-1])

IndexError: list assignment index out of range

In [257]:
muon_track = []
print(np.shape(track_list))

for i in range(len(track_list)):
    muon_track.append(track_list[i])
    print([i],'Injection point of the muon', muon_track[i].particle.pos)
    print('Length that the muon traveled in total', muon_track[i].particle.length)
    # Let's calculate the distance the muon traveled *inside the 800m volume*
    entering_large_cylinder = dataclasses.I3Position(muon_track[i].xi, muon_track[i].yi, muon_track[i].zi)
    print('Point where it enters the 800m volume', entering_large_cylinder)
    d_injection_cylinder = (muon_track[i].particle.pos - entering_large_cylinder).magnitude
    print('Distance from injection to 800m volume', d_injection_cylinder)
    d_inside = muon_track[i].particle.length - d_injection_cylinder

    # Ionization energy loss
    econstant = 1/4.5 # GeV/m
    e_ion = econstant*d_inside
    print('Ionization losses', e_ion)

    # Sum all the stochastics
    daughters = mctree.get_daughters(muon_track[i].particle)
    e_stochastic = 0
    for d in daughters:
        e_stochastic += d.energy

    print('Stochastic losses', e_stochastic)
    print('Total E lost inside volume', e_stochastic+e_ion)
    print('Energy available upon entering volume', muon_track[i].Ei)
    
    print('Entrance point of the particle (x,y,z)', dataclasses.I3Position(muon_track[i].xi, muon_track[i].yi, muon_track[i].zi))
    print('Length that the muon traveled in 800 (Length)', d_inside)

    daughters = mctree.get_daughters(muon_track[i].particle)


    e_stochastic = 0
    for d in daughters:
        position=d.pos
        if np.sqrt((position.x)**2 + (position.y)**2) <= 500.0 and  position.z>=-500.0 and position.z <= 500.0:
            e_stochastic += d.energy
    print('Stochastic losses in 500m',e_stochastic)


(2631,)


AttributeError: 'I3MMCTrackList' object has no attribute 'particle'